In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# read csv file
df = pd.read_csv('Load_history.csv')

# drop rows with NaN values
df.dropna(inplace=True)

# replace commas with empty strings and convert to integer
for i in range(1,25):
    var = 'h' + str(i)
    df[var] = df[var].str.replace(',', '').astype(int)

# split into features and target
X = df.iloc[:, :-24]  # features are all columns except the last 24
y = df.iloc[:, -24:]  # target is the last 24 columns

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.ensemble import RandomForestRegressor

# create model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# train model
rf_model.fit(X_train, y_train)
# rf_model.fit(X, y)

# make predictions
y_pred = rf_model.predict(X_test)


In [5]:
from sklearn.metrics import mean_absolute_error

# calculate MAE
mae = mean_absolute_error(y_test, y_pred)

print("Mean Absolute Error:", mae)

Mean Absolute Error: 6972.294033864025


In [8]:
# read new csv file
new_df = pd.read_csv('Benchmark.csv')

# split into features and target
X_new = new_df.iloc[:, 1:-24]  # features are all columns except the last 24
y_new = new_df.iloc[:, -24:]  # target is the last 24 columns

# make predictions
y_pred_new = rf_model.predict(X_new)

In [9]:
from sklearn.metrics import mean_absolute_error

# calculate MAE
mae = mean_absolute_error(y_new, y_pred_new)

print("Mean Absolute Error:", mae)

Mean Absolute Error: 88385.36602072309


#for months:

In [10]:
#for month
df['sum'] =0

# replace commas with empty strings and convert to integer
for i in range(1,25):
    var = 'h' + str(i)
    df['sum'] = df['sum'] + df[var]

print(df.head())
df_month = pd.DataFrame(columns= ['zone_id','year','month'])

x = 0
for i in range(2004,2009):
  for z in range(1,21): 
    for j in range(1,13):
      df_month.at[x,'year'] = i
      df_month.at[x,'zone_id'] = z
      df_month.at[x,'month'] = j
      x += 1

for i in range(1,31):
    var = 'd' + str(i)
    df_month[var] = 0


for d in range(1,32):
  for y in range(2004,2009):
    for m in range(1,13):
      for  z in  range(1, 21):      

                 df_subset = df.loc[(df['year'] == y) & (df['month'] == m) & (df['day'] == d) & (df['zone_id'] == z)]
                 df_subset = df_subset.dropna(subset=['sum'])  # drop empty cells in the 'sum' column
                 if not df_subset.empty:
                    df_month.loc[(df_month['year'] == y) & (df_month['month'] == m) & (df_month['zone_id'] == z), 'd' + str(d)] = int(df_subset['sum'])

#print(df_month.head())

df_month.to_csv("month.csv")

   zone_id  year  month  day     h1     h2     h3     h4     h5     h6  ...  \
0        1  2004      1    1  16853  16450  16517  16873  17064  17727  ...   
1        1  2004      1    2  14155  14038  14019  14489  14920  16072  ...   
2        1  2004      1    3  14439  14272  14109  14081  14775  15491  ...   
3        1  2004      1    4  11273  10415   9943   9859   9881  10248  ...   
4        1  2004      1    5  10750  10321  10107  10065  10419  12101  ...   

     h16    h17    h18    h19    h20    h21    h22    h23    h24     sum  
0  13138  14130  16809  18150  18235  17925  16904  16162  14750  400880  
1  15448  15839  17727  18895  18650  18443  17580  16467  15258  408157  
2  13414  13826  15825  16996  16394  15406  14278  13315  12424  366696  
3  13614  14162  16237  17430  17218  16633  15238  13580  11727  325465  
4  14350  15501  17307  18786  19089  19192  18416  17006  16018  347519  

[5 rows x 29 columns]
  zone_id  year month      d1      d2      d3      d

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

# Load the dataset
# Replace this with your own dataset or data preprocessing code
# Make sure to have your data in a format suitable for training an ANN
data = pd.read_csv('month.csv')
X = data.iloc[:, 0:4] # Features
y = data.iloc[:, 4:] # Target variables

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data preprocessing
# You may need to scale or normalize your data depending on the type of data you have
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the ANN model
model = Sequential()
# Add input layer
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
# Add hidden layers
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
# Add output layer with multiple units for multiple target variables
model.add(Dense(units=y_train.shape[1], activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
